In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pandas as pd
import matplotlib.pyplot as plt

from src.elasticity.elasticity import *
from src.optimization.optimization import *
from src.config.config import *


In [3]:
optimized_df = pd.read_csv(
    "../data/external/optimized_data.csv"
)

optimized_df.head()

,product_id,category,base_price,optimized_price,base_units,optimized_units,base_revenue,optimized_revenue,revenue_change_pct
0,cpvc_pipe__1_5_inch,Pipe,366.974893,376.85,940,933.27,346481.68,351706.30,0.0196
1,cpvc_pipe__1_inch,Pipe,256.317458,266.32,940,937.83,240745.58,249758.86,0.0366
2,cpvc_pipe__2_inch,Pipe,518.757004,528.75,968,968.24,501017.00,511957.91,0.0195
3,drip_irrigation_kit__0_5_acre,Irrigation,1207.939227,1217.89,989,987.20,1196261.85,1202303.61,0.0064
4,drip_irrigation_kit__1_acre,Irrigation,2023.752140,2033.72,939,937.39,1903357.40,1906392.22,0.0032


In [4]:
summary = {
    "base_revenue": optimized_df["base_revenue"].sum(),
    "optimized_revenue": optimized_df["optimized_revenue"].sum(),
    "revenue_uplift_pct": (
        optimized_df["optimized_revenue"].sum()
        / optimized_df["base_revenue"].sum()
        - 1
    )
}


In [6]:
summary

{'base_revenue': np.float64(160466937.70999998),
 'optimized_revenue': np.float64(161592063.69),
 'revenue_uplift_pct': np.float64(0.00701157506995842)}

In [7]:
assert (optimized_df["optimized_price"] >= optimized_df["base_price"]).all()
assert (optimized_df["optimized_units"] >= 0.95 * optimized_df["base_units"]).all()


In [8]:
(
    optimized_df
    .groupby("category")[["base_revenue", "optimized_revenue"]]
    .sum()
    .assign(
        uplift_pct=lambda x:
        x["optimized_revenue"] / x["base_revenue"] - 1
    )
)


,base_revenue,optimized_revenue,uplift_pct
category,,,
Fitting,6.831250e+05,7.141641e+05,0.045437
Irrigation,1.085556e+07,1.092373e+07,0.006280
Motor,1.436083e+08,1.445394e+08,0.006484
Pipe,5.319983e+06,5.414732e+06,0.017810


In [9]:
optimized_df["price_change"] = (
    optimized_df["optimized_price"] - optimized_df["base_price"]
)

optimized_df["price_change_pct"] = (
    optimized_df["price_change"] / optimized_df["base_price"]
)

optimized_df["unit_change_pct"] = (
    optimized_df["optimized_units"] / optimized_df["base_units"] - 1
)


In [10]:
optimized_df["risk_flag"] = (
    optimized_df["price_change_pct"].abs() > 0.08
)


In [11]:
final_output = optimized_df[
    [
        "product_id",
        "category",
        "base_price",
        "optimized_price",
        "price_change",
        "price_change_pct",
        "base_units",
        "optimized_units",
        "unit_change_pct",
        "base_revenue",
        "optimized_revenue",
        "revenue_change_pct"
    ]
]


In [12]:
final_output

,product_id,category,base_price,optimized_price,price_change,price_change_pct,base_units,optimized_units,unit_change_pct,base_revenue,optimized_revenue,revenue_change_pct
0,cpvc_pipe__1_5_inch,Pipe,366.974893,376.85,9.875107,0.026909,940,933.27,-0.007160,346481.68,351706.30,0.0196
1,cpvc_pipe__1_inch,Pipe,256.317458,266.32,10.002542,0.039024,940,937.83,-0.002309,240745.58,249758.86,0.0366
2,cpvc_pipe__2_inch,Pipe,518.757004,528.75,9.992996,0.019263,968,968.24,0.000248,501017.00,511957.91,0.0195
3,drip_irrigation_kit__0_5_acre,Irrigation,1207.939227,1217.89,9.950773,0.008238,989,987.20,-0.001820,1196261.85,1202303.61,0.0064
4,drip_irrigation_kit__1_acre,Irrigation,2023.752140,2033.72,9.967860,0.004925,939,937.39,-0.001715,1903357.40,1906392.22,0.0032
5,drip_irrigation_kit__2_acre,Irrigation,3513.373125,3523.30,9.926875,0.002825,944,944.10,0.000106,3300826.75,3326359.09,0.0029
6,foot_valve__1_inch,Fitting,407.554463,417.53,9.975537,0.024477,946,935.52,-0.011078,385121.47,390607.09,0.0131
7,motor_pump__0_5_hp_single_phase,Motor,10812.506109,11012.39,199.883891,0.018486,975,974.60,-0.000410,10544247.19,10732659.81,0.0181
8,motor_pump__1_5_hp_single_phase,Motor,17802.940333,18002.63,199.689667,0.011217,953,952.64,-0.000378,17113812.85,17149968.42,0.0108
9,motor_pump__1_hp_single_phase,Motor,14495.306992,14694.36,199.053008,0.013732,953,947.39,-0.005887,13769913.72,13921246.92,0.0078
